#### In this example, we download 100 listings from `wish.dim_all_products`. The steps are as follows:
1. Create an external table for the data export.
2. Execute a query to select some data into the external table.
3. Download the exported data from S3.

For downloading a large amount of data, this is much more efficient than directly running a SELECT query.

Additional tips:
* Use Hive for creating/altering/deleting tables and partitions. (You can't delete records by rows in Hive/Presto)
* Use Presto for SELECT (and INSERT SELECT). Don't use Presto to download more than a few thousand rows.


In [2]:
from tahoe import create_external_table, drop_external_table, execute_async
from s3 import result_bucket, get_s3_file_keys, get_df_from_parquet
import pandas as pd
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)s: %(message)s',
    level=logging.INFO
)

In [2]:
# Define the export table
export_table = {
    "name": "listing_export_test_123",
    "columns": [
        {"name": "listing_id", "type": "STRING"},
        {"name": "name", "type": "STRING"},
        {"name": "dt", "type": "STRING"},
    ]
}
db = "sweeper_dev"

In [3]:
# Create the export table
create_external_table(
    table_name=export_table["name"],
    table_definition=export_table,
    db=db,
    bucket=result_bucket
)

KeyError: 'TAHOE_USERNAME'

Data files in parquet format for this table will be stored in `'s3://wish-tahoe-query-results/sweeper_dev/listing_export_test_123'`

In [4]:
# Check the number of rows in the external table
q = f"SELECT COUNT(*) FROM {db}.{export_table['name']}"
execute_async(q)

2022-04-05 21:19:37,715 INFO: SELECT COUNT(*) FROM sweeper_dev.listing_export_test_123


[(0,)]

In [5]:
# Export data to the external table
q = f"""
INSERT INTO {db}.{export_table['name']}
SELECT 
    _id AS listing_id,
    name,
    dt
FROM wish.dim_all_products
LIMIT 1000
"""
execute_async(q)

2022-04-05 21:19:57,020 INFO: 
INSERT INTO sweeper_dev.listing_export_test_123
SELECT 
    _id AS listing_id,
    name,
    dt
FROM wish.dim_all_products
LIMIT 1000



[(1000,)]

In [6]:
# Check again the number of rows in the external table
q = f"SELECT COUNT(*) FROM {db}.{export_table['name']}"
execute_async(q)

2022-04-05 21:20:02,134 INFO: SELECT COUNT(*) FROM sweeper_dev.listing_export_test_123


[(1000,)]

In [9]:
# Show the data files for the external table
file_keys = get_s3_file_keys(s3_bucket=result_bucket, s3_prefix="sweeper_dev/listing_export_test_123")
file_keys

[('sweeper_dev/listing_export_test_123/20220406_021957_03711_w8227_ae1a4cff-f158-4399-b435-e47692608740',
  0),
 ('sweeper_dev/listing_export_test_123/20220406_021957_03711_w8227_d5c5b08a-6d32-455e-a9de-1435d359a095',
  0)]

In [13]:
# Import the parquet file as data frame
# Alternatively, you can use boto3 to directly download the file to disk
# For hundreds of data files, download using multiprocessing

dfs = []
for file_key, file_size in file_keys:
    df_chunk = get_df_from_parquet(s3_bucket=result_bucket, s3_key=file_key)
    dfs.append(df_chunk)
    
df_data = pd.concat(dfs)
df_data.head()

2022-04-05 21:21:49,252 INFO: Importing from sweeper_dev/listing_export_test_123/20220406_021957_03711_w8227_ae1a4cff-f158-4399-b435-e47692608740...
2022-04-05 21:21:50,296 INFO: Imported DF (349, 3) from sweeper_dev/listing_export_test_123/20220406_021957_03711_w8227_ae1a4cff-f158-4399-b435-e47692608740.
2022-04-05 21:21:50,344 INFO: Importing from sweeper_dev/listing_export_test_123/20220406_021957_03711_w8227_d5c5b08a-6d32-455e-a9de-1435d359a095...
2022-04-05 21:21:51,395 INFO: Imported DF (651, 3) from sweeper_dev/listing_export_test_123/20220406_021957_03711_w8227_d5c5b08a-6d32-455e-a9de-1435d359a095.


,listing_id,name,dt
0,62453dd22bab6cf58c61673f,Animal Pattern Temporary Tattoo Sticker Men Wa...,2022-03-31
1,62453fce324cee04c8b55caa,Gaiters Women's Stylish Leg Armor Covers Acces...,2022-03-31
2,6245402b70350f2946d94eee,Fashion Korea Silver Plated Demon Eye Finger R...,2022-03-31
3,62454101594260e9d77169ff,Men's / Women's Print 3D Shorts Sports Casual ...,2022-03-31
4,62454192881d18861dcd9b6a,All the Presidents' Gardens: Madison S Cabbage...,2022-03-31


In [14]:
df_data.to_parquet("sample_data.parquet", index=False)

In [15]:
# Optional: drop the external table
drop_external_table(
    db=db,
    table_name=export_table["name"],
    delete_files=True,
    s3_bucket=result_bucket,
    s3_prefix="sweeper_dev/listing_export_test_123", 
)

2022-04-05 21:22:20,997 INFO: USE `default`
2022-04-05 21:22:21,337 INFO: 
    DROP TABLE IF EXISTS sweeper_dev.listing_export_test_123
    
2022-04-05 21:22:21,890 INFO: The query returned no records.
2022-04-05 21:22:21,891 INFO: Dropped sweeper_dev.listing_export_test_123
2022-04-05 21:22:22,787 INFO: Files in 's3://wish-tahoe-query-results/sweeper_dev/listing_export_test_123 are deleted.


In [16]:
q = f"SELECT COUNT(*) FROM {db}.{export_table['name']}"
execute_async(q)

2022-04-05 21:22:29,042 INFO: SELECT COUNT(*) FROM sweeper_dev.listing_export_test_123
2022-04-05 21:22:31,551 WARNING: {'message': "line 1:22: Table 'hive.sweeper_dev.listing_export_test_123' does not exist", 'errorCode': 46, 'errorName': 'TABLE_NOT_FOUND', 'errorType': 'USER_ERROR', 'errorLocation': {'lineNumber': 1, 'columnNumber': 22}, 'failureInfo': {'type': 'io.prestosql.spi.PrestoException', 'message': "line 1:22: Table 'hive.sweeper_dev.listing_export_test_123' does not exist", 'suppressed': [], 'stack': ['io.prestosql.sql.analyzer.SemanticExceptions.semanticException(SemanticExceptions.java:48)', 'io.prestosql.sql.analyzer.SemanticExceptions.semanticException(SemanticExceptions.java:43)', 'io.prestosql.sql.analyzer.StatementAnalyzer$Visitor.visitTable(StatementAnalyzer.java:1229)', 'io.prestosql.sql.analyzer.StatementAnalyzer$Visitor.visitTable(StatementAnalyzer.java:329)', 'io.prestosql.sql.tree.Table.accept(Table.java:53)', 'io.prestosql.sql.tree.AstVisitor.process(AstVisito

DatabaseError: {'message': "line 1:22: Table 'hive.sweeper_dev.listing_export_test_123' does not exist", 'errorCode': 46, 'errorName': 'TABLE_NOT_FOUND', 'errorType': 'USER_ERROR', 'errorLocation': {'lineNumber': 1, 'columnNumber': 22}, 'failureInfo': {'type': 'io.prestosql.spi.PrestoException', 'message': "line 1:22: Table 'hive.sweeper_dev.listing_export_test_123' does not exist", 'suppressed': [], 'stack': ['io.prestosql.sql.analyzer.SemanticExceptions.semanticException(SemanticExceptions.java:48)', 'io.prestosql.sql.analyzer.SemanticExceptions.semanticException(SemanticExceptions.java:43)', 'io.prestosql.sql.analyzer.StatementAnalyzer$Visitor.visitTable(StatementAnalyzer.java:1229)', 'io.prestosql.sql.analyzer.StatementAnalyzer$Visitor.visitTable(StatementAnalyzer.java:329)', 'io.prestosql.sql.tree.Table.accept(Table.java:53)', 'io.prestosql.sql.tree.AstVisitor.process(AstVisitor.java:27)', 'io.prestosql.sql.analyzer.StatementAnalyzer$Visitor.process(StatementAnalyzer.java:346)', '

In [17]:
get_s3_file_keys(s3_bucket=result_bucket, s3_prefix="sweeper_dev/listing_export_test_123")

[]